In [2]:
import numpy as np
import pandas as pd
import queue
import matplotlib.pyplot as plt

from BayesianOptimization.bayesian_optimization import BayesianOptimization

from Backtest.backtest import Backtest
from Backtest.data import OHLCDataHandler
from ADX_BOLLStrategy import ADX_BOLLStrategy
from Backtest.open_json_gz_files import open_json_gz_files
from Backtest.generate_bars import generate_bars

In [10]:
def run_backtest(config, trading_data, ohlc_data, window_ADX, window_BOLL, a):
    window_ADX = int(window_ADX)
    window_BOLL = int(window_BOLL)
    config['title'] = "ADX_BOLLStrategy" + "_" + str(window_ADX) + "_" + str(window_BOLL) + "_" + str(a)
    print("---------------------------------")
    print(config['title'])
    print("---------------------------------")
    
    events_queue = queue.Queue()

    data_handler = OHLCDataHandler(
        config, events_queue,
        trading_data = trading_data, ohlc_data = ohlc_data
    )
    strategy = ADX_BOLLStrategy(config, events_queue, data_handler,
                           window_ADX = window_ADX,
                            window_BOLL = window_BOLL, a = a)

    backtest = Backtest(config, events_queue, strategy,
                        data_handler= data_handler)

    results = backtest.start_trading()
    
    return (results['cum_returns'][-1] - 1)

## Setting

In [11]:
config = {
    "csv_dir": "C:/backtest/Binance",
    "out_dir": "C:/backtest/results/ADX_BOLLStrategy",
    "title": "ADX_BOLLStrategy",
    "is_plot": False,
    "save_plot": False,
    "save_tradelog": False,
    "start_date": pd.Timestamp("2017-07-01T00:0:00", freq = "60" + "T"),    # str(freq) + "T"
    "end_date": pd.Timestamp("2018-04-01T00:00:00", freq = "60" + "T"),
    "equity": 1.0,
    "freq": 60,      # min
    "commission_ratio": 0.001,
    "suggested_quantity": None,     # None or a value
    "max_quantity": None,           # None or a value, Maximum purchase quantity
    "min_quantity": None,           # None or a value, Minimum purchase quantity
    "min_handheld_cash": None,      # None or a value, Minimum handheld funds
    "exchange": "Binance",
    "tickers": ['BTCUSDT']
}

In [12]:
# trading_data = {}
# for ticker in config['tickers']:
#     # trading_data[ticker] = open_gz_files(config['csv_dir'], ticker)
#     trading_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker + '.h5', key=ticker)

ohlc_data = {}
for ticker in config['tickers']:
    # ohlc_data[ticker] = generate_bars(trading_data, ticker, config['freq'])
    ohlc_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker +'_OHLC_60min.h5', key=ticker)

trading_data = None

In [13]:
gp_params = {"alpha": 1e-5}

## Acquisition Function "Upper Confidence Bound"

* Prefer exploitation (kappa=1.0)

* Prefer exploration (kappa=10)


## Acquisition Function "Expected Improvement"

* Prefer exploitation (xi=1e-4)
* Prefer exploration (xi=0.1)

In [14]:
BO = BayesianOptimization(
    run_backtest,
    {'window_ADX': (1, 240),
    'window_BOLL': (1, 240),
    'a': (0.1, 3)},
    is_int = [1, 1, 0], 
    invariant = {
        'config': config,
        'trading_data': trading_data,
        'ohlc_data': ohlc_data
    },
    random_state = 1
)
# BO.explore({
#     'short_window': np.arange(1, 120, 20),
#     'delta_window': np.arange(1, 120, 20)
#     },
#     eager=True)
BO.maximize(init_points=40, n_iter=60, acq="ei", xi=0.01, **gp_params)

Initialization
------------------------------------------------------------------------
 Step |   Time |      Value |         a |   window_ADX |   window_BOLL | 
---------------------------------
ADX_BOLLStrategy_38_23_0.7441611878525869
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.2473373025
Sortino Ratio: 2.0763982720
Max Drawdown: 0.3796025386
Max Drawdown Duration: 250900
Total Returns: 0.3254574378
Annualized Returns: 0.7659879515
Compound Annual Growth Rate: 0.9408718174
---------------------------------
Trades: 71
Trade Winning: 40.85%
Average Trade: 0.74%
Average Win: 4.91%
Average Loss: -2.14%
Best Trade: 26.23%
Worst Trade: -10.16%
Worst Trade Date: 0 days 02:00:00
Avg Days in Trade: 0 days 18:40:34
---

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.6530921023
Sortino Ratio: 1.6104334622
Max Drawdown: 0.2369021822
Max Drawdown Duration: 273500
Total Returns: 0.3445603105
Annualized Returns: 0.8109479635
Compound Annual Growth Rate: 1.0073501619
---------------------------------
Trades: 9
Trade Winning: 55.56%
Average Trade: 3.82%
Average Win: 8.68%
Average Loss: -2.26%
Best Trade: 18.76%
Worst Trade: -5.60%
Worst Trade Date: 1 days 06:00:00
Avg Days in Trade: 2 days 13:33:20
---------------------------------
   10 | 00m07s |    0.34456 | 2.740300 | 145.000000 | 116.000000 | 
---------------------------------
ADX_BOLLStrategy_130_176_0.9668973254081766
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
----------

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 2.1082098229
Sortino Ratio: 1.5485644593
Max Drawdown: 0.2453619175
Max Drawdown Duration: 273500
Total Returns: 0.4621405863
Annualized Returns: 1.0876817669
Compound Annual Growth Rate: 1.4451979606
---------------------------------
Trades: 5
Trade Winning: 80.00%
Average Trade: 8.49%
Average Win: 11.40%
Average Loss: -3.12%
Best Trade: 20.44%
Worst Trade: -3.12%
Worst Trade Date: 2 days 13:00:00
Avg Days in Trade: 4 days 15:48:00
---------------------------------
   19 | 00m08s |    0.46214 | 2.854821 | 102.000000 | 150.000000 | 
---------------------------------
ADX_BOLLStrategy_147_50_1.5298728212797632
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
----------

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 2.0014975447
Sortino Ratio: 2.5864863912
Max Drawdown: 0.2695822369
Max Drawdown Duration: 210400
Total Returns: 0.6418522832
Annualized Returns: 1.5106464268
Compound Annual Growth Rate: 2.2122146201
---------------------------------
Trades: 31
Trade Winning: 45.16%
Average Trade: 2.05%
Average Win: 7.69%
Average Loss: -2.59%
Best Trade: 24.37%
Worst Trade: -8.34%
Worst Trade Date: 1 days 13:00:00
Avg Days in Trade: 1 days 17:36:46
---------------------------------
   28 | 00m05s |    0.64185 | 1.668286 | 69.000000 | 53.000000 | 
---------------------------------
ADX_BOLLStrategy_216_81_1.3991425261192127
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
------------

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.7111047788
Sortino Ratio: 2.4264087024
Max Drawdown: 0.3279858490
Max Drawdown Duration: 273500
Total Returns: 0.5036170229
Annualized Returns: 1.1852996026
Compound Annual Growth Rate: 1.6115928594
---------------------------------
Trades: 53
Trade Winning: 39.62%
Average Trade: 1.11%
Average Win: 5.62%
Average Loss: -1.84%
Best Trade: 27.31%
Worst Trade: -7.54%
Worst Trade Date: 0 days 02:00:00
Avg Days in Trade: 0 days 18:56:36
---------------------------------
   37 | 00m04s |    0.50362 | 0.466313 | 138.000000 | 26.000000 | 
---------------------------------
ADX_BOLLStrategy_8_112_2.101593304489921
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
-------------

...

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.9000444400
Sortino Ratio: 0.9310136967
Max Drawdown: 0.1866709534
Max Drawdown Duration: 273500
Total Returns: 0.3391370208
Annualized Returns: 0.7981838533
Compound Annual Growth Rate: 0.9883460844
---------------------------------
Trades: 3
Trade Winning: 100.00%
Average Trade: 10.80%
Average Win: 10.80%
Average Loss: nan%
Best Trade: 22.63%
Worst Trade: 0.95%
Worst Trade Date: 2 days 16:00:00
Avg Days in Trade: 4 days 14:00:00
---------------------------------
   64 | 00m19s |    0.33914 | 3.000000 | 106.000000 | 240.000000 | 
---------------------------------
ADX_BOLLStrategy_240_240_0.1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
-------------------------

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.1765624834
Sortino Ratio: 0.7228586685
Max Drawdown: 0.1802914596
Max Drawdown Duration: 273500
Total Returns: 0.2170159548
Annualized Returns: 0.5107629672
Compound Annual Growth Rate: 0.5876363429
---------------------------------
Trades: 10
Trade Winning: 70.00%
Average Trade: 2.63%
Average Win: 6.88%
Average Loss: -7.28%
Best Trade: 23.29%
Worst Trade: -13.99%
Worst Trade Date: 0 days 09:00:00
Avg Days in Trade: 1 days 04:54:00
---------------------------------
   73 | 00m13s |    0.21702 | 2.716334 | 22.000000 | 79.000000 | 
---------------------------------
ADX_BOLLStrategy_167_74_3.0
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
--------------------------

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 2.6363755968
Sortino Ratio: 1.8496989851
Max Drawdown: 0.1974532332
Max Drawdown Duration: 172700
Total Returns: 0.5649067859
Annualized Returns: 1.3295495553
Compound Annual Growth Rate: 1.8690815213
---------------------------------
Trades: 4
Trade Winning: 100.00%
Average Trade: 12.41%
Average Win: 12.41%
Average Loss: nan%
Best Trade: 26.32%
Worst Trade: 3.11%
Worst Trade Date: 2 days 14:00:00
Avg Days in Trade: 4 days 12:00:00
---------------------------------
   82 | 00m18s |    0.56491 | 2.856214 | 80.000000 | 168.000000 | 
---------------------------------
ADX_BOLLStrategy_146_3_1.6409627303367202
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
-------------

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 2.0791565909
Sortino Ratio: 2.7958425833
Max Drawdown: 0.3295115023
Max Drawdown Duration: 273500
Total Returns: 0.6332360340
Annualized Returns: 1.4903674524
Compound Annual Growth Rate: 2.1726804898
---------------------------------
Trades: 45
Trade Winning: 31.11%
Average Trade: 1.57%
Average Win: 8.47%
Average Loss: -1.55%
Best Trade: 38.21%
Worst Trade: -5.68%
Worst Trade Date: 0 days 11:00:00
Avg Days in Trade: 1 days 00:10:40
---------------------------------
   91 | 00m15s |    0.63324 | 0.100000 | 225.000000 | 94.000000 | 
---------------------------------
ADX_BOLLStrategy_82_67_3.0
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.0647774442
Sortino Ratio: 1.0732251927
Max Drawdown: 0.4561441741
Max Drawdown Duration: 273500
Total Returns: 0.2321051000
Annualized Returns: 0.5462763772
Compound Annual Growth Rate: 0.6343540393
---------------------------------
Trades: 32
Trade Winning: 31.25%
Average Trade: 1.13%
Average Win: 9.77%
Average Loss: -2.80%
Best Trade: 23.90%
Worst Trade: -10.14%
Worst Trade Date: 2 days 04:00:00
Avg Days in Trade: 1 days 13:05:38
---------------------------------
  100 | 00m16s |    0.23211 | 0.100000 | 193.000000 | 116.000000 | 


In [15]:
print(BO.res['max'])

{'max_val': 0.9485894752027062, 'max_params': {'window_ADX': 26.0, 'window_BOLL': 72.0, 'a': 0.8568390408851148}}


In [16]:
BO.maximize(init_points=0, n_iter=60, acq="ei", xi=0.0001, **gp_params)

Bayesian Optimization
------------------------------------------------------------------------
 Step |   Time |      Value |         a |   window_ADX |   window_BOLL | 
---------------------------------
ADX_BOLLStrategy_215_95_0.1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.1305456293
Sortino Ratio: 1.2329685772
Max Drawdown: 0.4552628906
Max Drawdown Duration: 273500
Total Returns: 0.2641109557
Annualized Returns: 0.6216045061
Compound Annual Growth Rate: 0.7360367830
---------------------------------
Trades: 45
Trade Winning: 31.11%
Average Trade: 1.07%
Average Win: 8.28%
Average Loss: -2.19%
Best Trade: 46.58%
Worst Trade: -11.97%
Worst Trade Date: 1 days 02:00:00
Avg Days in Trade: 1 days 01:06:40
----------

---------------------------------
ADX_BOLLStrategy_176_1_0.1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
  110 | 00m11s |    0.00000 | 0.100000 | 176.000000 | 1.000000 | 
---------------------------------
ADX_BOLLStrategy_222_134_3.0
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 

---------------------------------
ADX_BOLLStrategy_135_132_3.0
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.9701062097
Sortino Ratio: 0.5294213593
Max Drawdown: 0.2812395145
Max Drawdown Duration: 273500
Total Returns: 0.1633185407
Annualized Returns: 0.3843821646
Compound Annual Growth Rate: 0.4276659157
---------------------------------
Trades: 4
Trade Winning: 50.00%
Average Trade: 4.47%
Average Win: 12.73%
Average Loss: -3.79%
Best Trade: 18.71%
Worst Trade: -5.60%
Worst Trade Date: 1 days 06:00:00
Avg Days in Trade: 3 days 22:30:00
---------------------------------
  119 | 00m18s |    0.16332 | 3.000000 | 135.000000 | 132.000000 | 
---------------------------------
ADX_BOLLStrategy_220_18_0.5023246155019785


---------------------------------
ADX_BOLLStrategy_35_240_0.1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.2189530535
Sortino Ratio: 1.7521769539
Max Drawdown: 0.3445243390
Max Drawdown Duration: 273500
Total Returns: 0.2972319665
Annualized Returns: 0.6995572344
Compound Annual Growth Rate: 0.8449954954
---------------------------------
Trades: 66
Trade Winning: 33.33%
Average Trade: 0.74%
Average Win: 5.81%
Average Loss: -1.79%
Best Trade: 29.36%
Worst Trade: -6.97%
Worst Trade Date: 0 days 10:00:00
Avg Days in Trade: 0 days 18:00:00
---------------------------------
  128 | 00m24s |    0.29723 | 0.100000 | 35.000000 | 240.000000 | 
---------------------------------
ADX_BOLLStrategy_97_63_3.0
------------------

---------------------------------
ADX_BOLLStrategy_58_152_3.0
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 2.3651732552
Sortino Ratio: 1.4930175626
Max Drawdown: 0.2182973575
Max Drawdown Duration: 192900
Total Returns: 0.4186640500
Annualized Returns: 0.9853565498
Compound Annual Growth Rate: 1.2775068549
---------------------------------
Trades: 4
Trade Winning: 100.00%
Average Trade: 9.52%
Average Win: 9.52%
Average Loss: nan%
Best Trade: 19.60%
Worst Trade: 3.25%
Worst Trade Date: 2 days 00:00:00
Avg Days in Trade: 3 days 20:15:00
---------------------------------
  137 | 00m23s |    0.41866 | 3.000000 | 58.000000 | 152.000000 | 
---------------------------------
ADX_BOLLStrategy_240_17_3.0
--------------------

---------------------------------
ADX_BOLLStrategy_201_73_3.0
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -0.3826845756
Sortino Ratio: -0.1974077518
Max Drawdown: 0.3418648477
Max Drawdown Duration: 273500
Total Returns: -0.1503634444
Annualized Returns: -0.3538913952
Compound Annual Growth Rate: -0.3185325429
---------------------------------
Trades: 6
Trade Winning: 33.33%
Average Trade: -1.88%
Average Win: 10.50%
Average Loss: -8.07%
Best Trade: 17.48%
Worst Trade: -17.28%
Worst Trade Date: 0 days 06:00:00
Avg Days in Trade: 2 days 18:50:00
---------------------------------
  146 | 00m17s |   -0.15036 | 3.000000 | 201.000000 | 73.000000 | 
---------------------------------
ADX_BOLLStrategy_35_1_0.1
------------

---------------------------------
ADX_BOLLStrategy_197_222_3.0
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.0906437008
Sortino Ratio: 0.4900544230
Max Drawdown: 0.2253388824
Max Drawdown Duration: 273500
Total Returns: 0.1829048286
Annualized Returns: 0.4304799297
Compound Annual Growth Rate: 0.4848846432
---------------------------------
Trades: 3
Trade Winning: 66.67%
Average Trade: 6.38%
Average Win: 11.49%
Average Loss: -3.85%
Best Trade: 18.84%
Worst Trade: -3.85%
Worst Trade Date: 2 days 17:00:00
Avg Days in Trade: 5 days 11:00:00
---------------------------------
  155 | 00m24s |    0.18290 | 3.000000 | 197.000000 | 222.000000 | 
---------------------------------
ADX_BOLLStrategy_142_240_3.0
--------------

In [17]:
print(BO.res['max'])

{'max_val': 0.9485894752027062, 'max_params': {'window_ADX': 26.0, 'window_BOLL': 72.0, 'a': 0.8568390408851148}}


In [18]:
Target = pd.DataFrame({'Parameters': BO.X.tolist(), 'Target': BO.Y})
# Target.to_csv(config['out_dir'] + "/target_ei.csv")
Target.sort_values(by = "Target")

Parameters    Target
139                    [1.0, 94.0, 0.1] -0.731121
102                   [1.0, 240.0, 0.1] -0.671160
58                    [1.0, 169.0, 0.1] -0.628136
50     [2.0, 68.0, 0.13916459812046364] -0.479027
70     [4.0, 156.0, 0.7836778791234862] -0.433329
115    [200.0, 5.0, 0.8523102770647875] -0.209054
73                   [167.0, 74.0, 3.0] -0.190399
108                  [220.0, 20.0, 0.1] -0.156681
141                 [219.0, 196.0, 0.1] -0.151760
145                  [201.0, 73.0, 3.0] -0.150363
112                  [27.0, 180.0, 3.0] -0.095223
11   [205.0, 235.0, 1.7947284501399945] -0.067155
149   [13.0, 218.0, 2.9192147457614697] -0.065104
55                  [159.0, 240.0, 0.1] -0.062882
86                  [214.0, 175.0, 0.1] -0.061370
105    [81.0, 64.0, 2.7535145584960072] -0.056274
123                   [41.0, 39.0, 3.0] -0.054133
94      [8.0, 73.0, 0.8129170829906152] -0.048706
47                    [240.0, 1.0, 0.1]  0.000000
62                    [106.0, 1.0, 0.1]  0.000000
129     [221.0, 1.0, 2.454165609901493]  0.000000
46                      [1.0, 1.0, 3.0]  0.000000
21      [140.0, 4.0, 2.319906802878526]  0.000000
41                     [64.0, 1.0, 0.1]  0.000000
146                    [35.0, 1.0, 0.1]  0.000000
103                    [84.0, 1.0, 3.0]  0.000000
2      [141.0, 2.0, 1.4607917266706152]  0.000000
97                    [200.0, 1.0, 0.1]  0.000000
82     [146.0, 3.0, 1.6409627303367202]  0.000000
109                   [176.0, 1.0, 0.1]  0.000000
..                                  ...       ...
36    [138.0, 26.0, 0.4663131666301761]  0.503617
88                    [96.0, 48.0, 0.1]  0.510972
56                   [240.0, 35.0, 3.0]  0.512065
48                    [41.0, 58.0, 3.0]  0.512655
33     [87.0, 16.0, 1.1476569586137009]  0.515689
98                   [85.0, 107.0, 0.1]  0.523792
20    [213.0, 58.0, 1.6677414755885676]  0.524405
135   [138.0, 26.0, 1.3843488578437746]  0.564537
81    [80.0, 168.0, 2.8562143900078505]  0.564907
17    [21.0, 199.0, 1.6470006227550085]  0.575348
137                  [240.0, 17.0, 3.0]  0.578689
140   [58.0, 38.0, 0.12344401630206449]  0.580053
28    [216.0, 81.0, 1.3991425261192127]  0.589008
19    [147.0, 50.0, 1.5298728212797632]  0.604063
114                 [156.0, 133.0, 0.1]  0.605768
78                  [240.0, 137.0, 0.1]  0.622113
66    [69.0, 226.0, 1.8940577816927975]  0.622470
90                   [225.0, 94.0, 0.1]  0.633236
27     [69.0, 53.0, 1.6682857498427974]  0.641852
22   [235.0, 197.0, 0.2315026128664944]  0.661780
26     [51.0, 27.0, 2.6610638798763273]  0.678268
30   [234.0, 116.0, 1.1949945893381744]  0.685829
10   [130.0, 176.0, 0.9668973254081766]  0.687310
31    [223.0, 42.0, 1.6614316134119567]  0.696577
131   [82.0, 45.0, 0.16666915645142422]  0.735318
45    [231.0, 52.0, 1.0831337440623408]  0.754338
32    [97.0, 211.0, 1.9916667579842722]  0.757219
143                  [28.0, 107.0, 0.1]  0.802480
65                    [51.0, 41.0, 0.1]  0.807736
15     [26.0, 72.0, 0.8568390408851148]  0.948589

[160 rows x 2 columns]

## Acquisition Function "Probability of Improvement"

* Prefer exploitation (xi=1e-4)
* Prefer exploration (xi=0.1)